<a href="https://colab.research.google.com/github/Darshan-Thakar/NLP_and_ML_projects/blob/main/CyberbullyingClassification2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import re
import spacy
from tensorflow import keras
import tensorflow as tf

In [ ]:
df=pd.read_csv("cyberbullying_tweets.csv");

In [ ]:
df.head(4)

,tweet_text,cyberbullying_type
0,"In other words #katandandre, your food was cra...",not_cyberbullying
1,Why is #aussietv so white? #MKR #theblock #ImA...,not_cyberbullying
2,@XochitlSuckkks a classy whore? Or more red ve...,not_cyberbullying
3,"@Jason_Gio meh. :P thanks for the heads up, b...",not_cyberbullying


In [ ]:
df.cyberbullying_type.value_counts()

religion               7998
age                    7992
gender                 7973
ethnicity              7961
not_cyberbullying      7945
other_cyberbullying    7823
Name: cyberbullying_type, dtype: int64

In [ ]:
df=df[df['cyberbullying_type']!='other_cyberbullying'];

In [ ]:
df.cyberbullying_type.value_counts()

religion             7998
age                  7992
gender               7973
ethnicity            7961
not_cyberbullying    7945
Name: cyberbullying_type, dtype: int64

In [ ]:
nlp=spacy.load("en_core_web_sm");

In [ ]:
def preprocess(text):
  text=re.sub('[^\w\s]',' ',text);
  text=re.sub(' +',' ',text);
  text=text.strip().lower();
  doc=nlp(text);
  filtered_tokens=[token.lemma_ for token in doc if not token.is_stop];
  return " ".join(filtered_tokens);


In [ ]:
df.head(4)

,tweet_text,cyberbullying_type
0,"In other words #katandandre, your food was cra...",not_cyberbullying
1,Why is #aussietv so white? #MKR #theblock #ImA...,not_cyberbullying
2,@XochitlSuckkks a classy whore? Or more red ve...,not_cyberbullying
3,"@Jason_Gio meh. :P thanks for the heads up, b...",not_cyberbullying


In [ ]:
df['preprocessed_text']=df.tweet_text.apply(preprocess);

In [ ]:
df['cyberbullying_type']="__label__" + df['cyberbullying_type'].astype(str);

In [ ]:
df['fasttext_text']=df['cyberbullying_type'] + " " + df['preprocessed_text'];

In [ ]:
from sklearn.model_selection import train_test_split

X_train,X_test=train_test_split(df['fasttext_text'],test_size=0.2,random_state=1000);

In [ ]:
X_train.to_csv("fasttext_training.csv",header=False,index=False);
X_test.to_csv("fasttext_testing.csv",header=False, index=False);


In [ ]:
! pip install fasttext

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 68 kB 4.7 MB/s 
  Using cached pybind11-2.10.1-py3-none-any.whl (216 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp37-cp37m-linux_x86_64.whl size=3158078 sha256=96a3f11eb152bb4e9761cb4c2330adb08a78a0e1a986ab2a9093c20b96e3f8ad
  Stored in directory: /root/.cache/pip/wheels/4e/ca/bf/b020d2be95f7641801a6597a29c8f4f19e38f9c02a345bab9b
Successfully built fasttext


In [ ]:
import fasttext;

In [ ]:
model=fasttext.train_supervised(input="fasttext_training.csv");
model.test("fasttext_testing.csv")

(9535, 0.8384897745149449, 0.8384897745149449)

In [ ]:
# It is performing not that good

# Let's try lstm and gru's


In [ ]:
vocab_size=1000000;

one_hot_encoded=[keras.preprocessing.text.one_hot(d,vocab_size) for d in df.preprocessed_text];
print(one_hot_encoded);

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:

max_length=60;
padded_reviews=keras.preprocessing.sequence.pad_sequences(one_hot_encoded,padding='post');

In [ ]:
print(df.cyberbullying_type.unique())
print(padded_reviews.shape)

['__label__not_cyberbullying' '__label__gender' '__label__religion'
 '__label__age' '__label__ethnicity']
(39869, 220)


In [ ]:
target=df.cyberbullying_type.map({'__label__not_cyberbullying':0,'__label__gender':1,'__label__religion':2,'__label__age':3,'__label__ethnicity':4});

In [ ]:
from sklearn.model_selection import train_test_split

X_train,X_test,Y_train,Y_test=train_test_split(padded_reviews,target,test_size=0.19,random_state=1000);


In [ ]:

model=keras.Sequential([
    keras.layers.Embedding(vocab_size,120),
    
    keras.layers.Bidirectional(keras.layers.GRU(15,activation="tanh")),
   
    keras.layers.Dense(5,activation='softmax')
])

model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)


In [ ]:
model.fit(X_train,Y_train,epochs=17,)

Epoch 1/17
1010/1010 [==============================] - 82s 74ms/step - loss: 0.4078 - accuracy: 0.8707
Epoch 2/17
1010/1010 [==============================] - 75s 75ms/step - loss: 0.1278 - accuracy: 0.9607
Epoch 3/17
1010/1010 [==============================] - 75s 74ms/step - loss: 0.0656 - accuracy: 0.9800
Epoch 4/17
1010/1010 [==============================] - 75s 74ms/step - loss: 0.0370 - accuracy: 0.9893
Epoch 5/17
1010/1010 [==============================] - 75s 74ms/step - loss: 0.0247 - accuracy: 0.9925
Epoch 6/17
1010/1010 [==============================] - 75s 74ms/step - loss: 0.0178 - accuracy: 0.9945
Epoch 7/17
1010/1010 [==============================] - 75s 74ms/step - loss: 0.0144 - accuracy: 0.9954
Epoch 8/17
1010/1010 [==============================] - 75s 74ms/step - loss: 0.0103 - accuracy: 0.9966
Epoch 9/17
1010/1010 [==============================] - 75s 74ms/step - loss: 0.0085 - accuracy: 0.9970
Epoch 10/17
1010/1010 [==============================] - 75s 74m

In [ ]:
from google.colab import drive

drive.mount('/content/drive');

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pickle

In [ ]:
with open("/content/drive/MyDrive/GRU_effonet","wb") as fb:
  pickle.dump(model,fb);

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
y_pred=model.predict(X_test);


237/237 [==============================] - 2s 6ms/step


In [ ]:
y_pred=[np.argmax(i) for i in y_pred];

In [ ]:
cr_gru=classification_report(Y_test,y_pred);
print(cr_gru)

              precision    recall  f1-score   support

           0       0.80      0.75      0.78      1509
           1       0.84      0.88      0.86      1540
           2       0.93      0.95      0.94      1497
           3       0.95      0.95      0.95      1510
           4       0.98      0.97      0.97      1520

    accuracy                           0.90      7576
   macro avg       0.90      0.90      0.90      7576
weighted avg       0.90      0.90      0.90      7576



In [ ]:
corpus=['I am very tired'];
test_encoded=[keras.preprocessing.text.one_hot(d,vocab_size) for d in corpus];

padded_test=keras.preprocessing.sequence.pad_sequences(test_encoded,padding='post');

In [ ]:
sample_prediction=model.predict(padded_test);

1/1 [==============================] - 2s 2s/step


In [ ]:
np.argmax(sample_prediction)

0